# Transformación geométrica de una imagen

## Objetivo
En este ejemplo, se aborda la transformación geométrica interactiva de una imagen, permitiendo ajustar parámetros como traslación, escalado, rotación, sesgo afín y proyectividad. Para su solución, se utilizan sliders interactivos que controlan estos parámetros en tiempo real, aplicando las transformaciones mediante matrices geométricas. Los valores de los sliders se combinan en una única matriz de transformación que se aplica a la imagen usando cv2.warpPerspective, mostrando los resultados de forma inmediata. Los límites conservadores en las rotaciones, sesgos y proyectividades evitan deformaciones extremas, garantizando una manipulación controlada de la imagen.


## Carga de la imagen
Cargamos la imagen en formato BGR utilizando cv2.imread() y la convertimos a RGB con cv2.cvtColor() para asegurar que se muestre correctamente en las librerías de visualización como matplotlib.

In [112]:
!rm Alejandro_con_Paul.jpg
!wget https://maltimor.com/Alejandro_con_Paul.jpg  # Descarga foto


--2024-10-31 19:11:38--  https://maltimor.com/Alejandro_con_Paul.jpg
Resolving maltimor.com (maltimor.com)... 217.76.156.72
Connecting to maltimor.com (maltimor.com)|217.76.156.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008960 (1.9M) [image/jpeg]
Saving to: ‘Alejandro_con_Paul.jpg’

Alejandro_con_Paul. 100%[===================>]   1.92M  2.50MB/s    in 0.8s    

2024-10-31 19:11:39 (2.50 MB/s) - ‘Alejandro_con_Paul.jpg’ saved [2008960/2008960]



In [113]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import ipywidgets as widgets
from IPython.display import display

# Cargar la imagen en color
img = cv2.imread('Alejandro_con_Paul.jpg')
if img is None:
    raise FileNotFoundError("No se pudo encontrar la imagen. Verifica la ruta.")

# Convertir la imagen de BGR (formato de OpenCV) a RGB para mostrarla correctamente con matplotlib
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


## Función de transformación matricial
Definimos una función **apply_transformations(tx, ty, sx, sy, alfa, a, p1, p2)**, que realiza las siguientes operaciones:

- **Traslación** (T): Desplaza la imagen en los ejes x e y según los valores de tx y ty.
- **Escalado** (S): Redimensiona la imagen de forma independiente en los ejes x e y, utilizando los factores de escalado sx y sy.
- **Rotación** (R): Gira la imagen en torno a la esquina superior izquierda, según el ángulo alfa (convertido a radianes).
- **Sesgo afín** (A): Aplica una inclinación o sesgo a la imagen, controlado por el parámetro a.
- **Proyectividad** (P): Modifica la perspectiva de la imagen mediante los parámetros de proyectividad p1 y p2, lo que afecta la distorsión proyectiva.

La matriz de transformación combinada M es el resultado de multiplicar todas las matrices en orden. La transformación completa se aplica a la imagen utilizando cv2.warpPerspective(), que genera la imagen transformada.

In [114]:
# Función para aplicar las transformaciones geométricas
def apply_transformations(tx, ty, sx, sy, alfa, a, p1, p2):

    h, w = img.shape[:2]

    # Convertir ángulo a radianes
    alfa_rad = np.deg2rad(alfa)

    # Matriz K (normalización)
    K = np.array([[1, 0, w/2],
                  [0, 1, h/2],
                  [0, 0, 1]])

    # Matriz de rotación y escalado (S)
    S = np.array([[sx * np.cos(alfa_rad), -sx * np.sin(alfa_rad), tx],
                  [sy * np.sin(alfa_rad), sy * np.cos(alfa_rad), ty],
                  [0, 0, 1]])

    # Matriz de sesgo afín (A)
    A = np.array([[1, a, 0],
                  [0, 1, 0],
                  [0, 0, 1]])

    # Matriz de proyectividad (P)
    P = np.array([[1, 0, 0],
                  [0, 1, 0],
                  [p1, p2, 1]])

    # Matriz de transformación combinada K * S * A * P
#    H =  K @ S @ A @ P
    H =  S @ A @ P
    # Aplicar la transformación a la imagen
    transformed_img = cv2.warpPerspective(img, H, (w, h))

    return transformed_img


## Parametrización interactiva
La función **update** aplica las transformaciones geométricas utilizando los valores de los sliders, que controlan los parámetros de traslación, escalado, rotación, sesgo y proyectividad. Los sliders interactivos permiten ajustar cada transformación y actualizar la imagen visualizada en la interfaz con límites conservadores para evitar distorsiones extremas.

In [115]:
# Función para actualizar la imagen con sliders
def update(tx=0, ty=0, sx=1, sy=1, alfa=0, a=0, p1=0, p2=0):
    transformed_img = apply_transformations(tx, ty, sx, sy, alfa, a, p1, p2)
    plt.imshow(transformed_img)
    plt.axis('off')
    plt.show()

# Crear los sliders interactivos
tx_slider = widgets.FloatSlider(value=0, min=-200, max=200, step=1, description='Traslación X')
ty_slider = widgets.FloatSlider(value=0, min=-200, max=200, step=1, description='Traslación Y')
sx_slider = widgets.FloatSlider(value=1, min=0.8, max=1.2, step=0.01, description='Escalado X')
sy_slider = widgets.FloatSlider(value=1, min=0.8, max=1.2, step=0.01, description='Escalado Y')
rot_slider = widgets.FloatSlider(value=0, min=-45, max=45, step=1, description='Rotación (°)')
skew_slider = widgets.FloatSlider(value=0, min=-0.3, max=0.3, step=0.01, description='Sesgo afín')
p1_slider = widgets.FloatSlider(value=0, min=-0.001, max=0.001, step=0.0001, description='Proyectividad P1')
p2_slider = widgets.FloatSlider(value=0, min=-0.001, max=0.001, step=0.0001, description='Proyectividad P2')

# Mostrar los sliders y la imagen transformada
ui = widgets.VBox([tx_slider, ty_slider, sx_slider, sy_slider, rot_slider, skew_slider, p1_slider, p2_slider])

# Conectar los sliders con la función de actualización
out = widgets.interactive_output(update, {'tx': tx_slider, 'ty': ty_slider, 'sx': sx_slider, 'sy': sy_slider,
                                          'alfa': rot_slider, 'a': skew_slider, 'p1': p1_slider, 'p2': p2_slider})

# Mostrar la interfaz
display(ui, out)


Output()